In [11]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import quandl
import math
from datetime import datetime
token="Us3wFmXGgAj_1cUtHAAR"

In [3]:
import pylab
sns.set(font_scale=3)
pylab.rcParams['figure.figsize'] = (24, 18)

# Load Data from quandl

* Select a few different contracts
* Need to get contract sizes (or do we?)

In [ ]:
df=pd.DataFrame()
df['SP500']=quandl.get("CHRIS/CME_SP1", authtoken=token).Settle
df['Copper']=quandl.get("CHRIS/CME_HG1", authtoken=token).Settle
df['Gold']=quandl.get("CHRIS/CME_GC1", authtoken=token).Settle

# Signal

Moskowitz et al (2013): http://docs.lhpedersen.com/TimeSeriesMomentum.pdf

But we also need to scale each position.  As per the paper this is done by in volatility.

$$ signal = \frac{sign[r_{t-12,t}^s]}{\sigma_t^s} $$


In [4]:
def tsmom_daily(data,signal_lookback,vol_lookback=20):
	mul=get_contract_multipliers()[data.columns]
	vol=pd.ewmstd(data,vol_lookback,min_periods=vol_lookback)*math.sqrt(256)
	signal=pd.rolling_mean(data,signal_lookback)
	signal = signal /abs(signal)
	position=(signal / (vol *mul))
	return position.shift(1)

In [5]:
def calc_pnl_wc(position,data,fee=0.0005):
	mul=get_contract_multipliers()[data.columns]
	return position*data*mul-cost_model(position,fee)

# Scaling factor

Need to scale the portfolio to hit expected volatility.

$$ ScalingFactor = \frac{Target}{Expected[vol]}$$

Hard part, how do you estimate expected vol.  

$$ RealizedVol = w' \Sigma w $$

In [6]:
# Has a bit of hindsight bias involved since it uses entire time for Covariance matrixd
def calc_scaling_factor(pnl,vol_target=0.2):
    Sigma_all=pnl.cov().dropna(how='all',axis=1).dropna(how='all')
    ind=pnl.dropna(how='all').index
    SF=[]
    for row in pnl.dropna(how='all').iterrows():
        no_mkts=row[1].dropna().count()
        w=np.array([1/float(no_mkts)]*no_mkts)
        mkt_list=row[1].dropna().index
        Sig = Sigma_all[mkt_list].T[mkt_list]
        vol_ach=np.sqrt(np.dot(np.dot(w.T,Sig.as_matrix()),w))*16
        SF.append(vol_target/vol_ach)
    return pd.Series(SF,index=ind)

# Calculate Portfolio PNL 

$$ PNL_{Portfolio} = \sum_i^n \frac{1}{N}  PNL_i * ScalingFactor $$

In [ ]:
#remove_inf(pnl).mean(axis=1).dropna()*scaling_factor